# <span style="color:blue">Vehicle Detection Software</span>.

Project Prepared By:
1. Youssef Abdelaziz Farrag ID : 1701732
2. Merna Saeed Habib ID : 1701526

This software was done using the following libraries:
- OpenCV
- Matplotlib
- Numpy
- MoviePy

## Program Pipeline:

1. HOG Feature Extraction on training data
2. Training SVM Classifier
3. Sliding Windows Technique
4. Heatmap and Thresholding

## HOG Feature Extraction

In the first phase of the project, we perform HOG feature extraction on a set of labeled test images, labeled as vehicles and non-vehicles. 

<table>
    <th><img src="kiti1.png" width = 150></th>
    <th><img src="kiti2.png" width = 150></th>
    <th><img src="kiti3.png" width = 150></th>
</table>

<center> HOG Feature Extraction </center>

<table>
    <th><img src="carex1.png" width = 150></th>
    <th><img src="carex2.png" width = 150></th>
    <th><img src="carex3.png" width = 150></th>
</table>

<center> Examples of cars from the training set of images.</center>

<table>
    <th><img src="nocarex1.png" width = 150></th>
    <th><img src="nocarex2.png" width = 150></th>
    <th><img src="nocarex3.png" width = 150></th>
</table>

<center> Examples of non-vehicles in the training set of images </center>

This is done by the following function:

In [1]:
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    ''' Define a function to extract features from a list of images
    This function call get_hog_features(), bin_spatial() and color_hist()
    '''
    # Create a list to append feature vectors to
    vector_features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        
        # Apply color conversion if other than 'RGB'
        feature_image = convert_RGB_color(image, color_space = color_space)     

        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        
        if spatial_feat == True:
            # Apply bin_spatial()
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        
        vector_features.append(np.concatenate(file_features))
    
    # Return list of feature vectors
    return vector_features

****

## Training the SVM Classifier

In this phase, we train the SVM Classifier using the HOG features we obtained from the first phase, and using a YCrCb Color space for more accuracy. We used only 2760 images for training the SVM Classifier for better memory and speed.

****

## Sliding Window Search

In this phase, we also implement a sliding window search as we did in the lane detection phase. The sliding window technique is the most commonly used method of pixel detection in image processing.

The problem in the sliding window search is that it consumes so much time, so we start the scaling of the image from 0.8 to 1.5 . The window then slides from top to bottom and left to right. The windows overlap to increase accuracy. It then selects the rectangles with a minimum number of pixels inside.

In [2]:
def find_cars(img, ystart, ystop, scale, svc, X_scaler, 
              orient, pix_per_cell, cell_per_block, spatial_size, hist_bins,
              color_space,hog_channel,cells_per_step,idx,spatial_feat=True,hist_feat=False,clip = False):
    
    bbox_list=[]
    out_img = np.copy(img)
    
    if clip:
        img = img.astype(np.float32)/255  
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_RGB_color(img_tosearch, color_space=color_space)    
    
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    if (hog_channel=='ALL'):
        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]
    else:
        ch1 = ctrans_tosearch[:,:,hog_channel]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    
    nblocks_per_window = (window // pix_per_cell)-1 
    
    #cells_per_step = 2  # Instead of overlap, define how many cells to step (2)
        
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    if (hog_channel=='ALL'):
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    #count = 0
    for xb in range(nxsteps):
        for yb in range(nysteps):
            #count +=1
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            if (hog_channel=='ALL'):
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            else:
                hog_features = np.hstack((hog_feat1))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            if spatial_feat:
                spatial_features = bin_spatial(subimg, size=spatial_size)
            if hist_feat:
                hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            if hist_feat:
                test_features = X_scaler.transform(np.hstack((
                    hist_features,hog_features,spatial_features)).reshape(1, -1)) 
            else:
                temp = np.hstack((hog_features,spatial_features)).reshape(1, -1)
                temp = reduce_features(temp,idx)
                test_features = X_scaler.transform(temp) 
            
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(out_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,1.),6) 
                bbox_list.append([(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)])
    
    
    return out_img,bbox_list

****

## Heatmap

In this phase, we use a heatmap to identify the areas with the most concentration of pixels.

In [3]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    # Return updated heatmap
    return heatmap

After we identify the areas with most recurring pixels, we threshold the image:

In [4]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

<table>
    <th><img src="heatmap1.jpg" width = 400></th>
    <th><img src="heatmap2.jpg" width = 400></th>
</table>

<center> Heatmap before and after thresholding </center>

****

## Detection of Vehicles and Bounding Box

After the previous operations are done, and we have a heatmap of recurring pixels, we start by drawing a bounding box on the pixels detected, after that we average out nearby boxes and draw one big box:

In [5]:
def draw_boxes(img, bboxes, color=(0., 0., 1.), thick=6):
    ''' Define a function to draw bounding boxes'''
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

<center><img src='rawbox.jpg' width=500></center>

<center> Raw detection before averaging the boxes </center>

<center><img src='final.jpg' width=500></center>

<center> Final car position, after removing the unwanted pixels and averaging the boxes </center>

****